In [1]:
import torch
import numpy as np
import os
from PIL import Image
from IPython.display import display



In [2]:
with open('train/train-images-idx3-ubyte', 'rb') as train:
    magic = int.from_bytes(train.read(4), byteorder='big')
    N = int.from_bytes(train.read(4), byteorder='big')
    row_size = int.from_bytes(train.read(4), byteorder='big')
    col_size = int.from_bytes(train.read(4), byteorder='big')
    assert(magic == 2051)
    
    training_set = np.empty((N,row_size,col_size), dtype='uint8')
    
    for i in range(N):
        data = train.read(row_size*col_size)
        img = np.frombuffer(data, dtype='ubyte').reshape((row_size,col_size))
        training_set[i] = img

In [3]:
with open('train/train-labels-idx1-ubyte', 'rb') as labels:
    magic = int.from_bytes(labels.read(4), byteorder='big')
    N = int.from_bytes(labels.read(4), byteorder='big')
    assert(magic == 2049)
    
    training_labels = np.empty(N, dtype='uint8')
    
    for i in range(N):
        training_labels[i] = int.from_bytes(labels.read(1), byteorder='big')


In [4]:
with open('test/t10k-images-idx3-ubyte', 'rb') as test:
    magic = int.from_bytes(test.read(4), byteorder='big')
    test_N = int.from_bytes(test.read(4), byteorder='big')
    row_size = int.from_bytes(test.read(4), byteorder='big')
    col_size = int.from_bytes(test.read(4), byteorder='big')
    assert(magic == 2051)
    
    test_set = np.empty((test_N,row_size,col_size), dtype='uint8')
    
    for i in range(test_N):
        data = test.read(row_size*col_size)
        img = np.frombuffer(data, dtype='ubyte').reshape((row_size,col_size))
        test_set[i] = img

In [5]:
with open('test/t10k-labels-idx1-ubyte', 'rb') as labels:
    magic = int.from_bytes(labels.read(4), byteorder='big')
    test_N = int.from_bytes(labels.read(4), byteorder='big')
    assert(magic == 2049)
    
    test_labels = np.empty(test_N, dtype='uint8')
    
    for i in range(test_N):
        test_labels[i] = int.from_bytes(labels.read(1), byteorder='big')


In [10]:
sample = 8930
display(Image.fromarray(training_set[sample]))
print(training_labels[sample])

4


In [13]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(1,16,kernel_size=3,stride=2,padding=1),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(16,32,kernel_size=2,stride=1),
    torch.nn.ReLU(inplace=True),
    torch.nn.Conv2d(32,128,kernel_size=2,stride=1),
    torch.nn.ReLU(inplace=True),
    torch.nn.BatchNorm2d(128),
    torch.nn.Flatten(),
    torch.nn.Linear(18432, 1024),
    torch.nn.Dropout(),
    torch.nn.Sigmoid(),
    torch.nn.Linear(1024, 256),
    torch.nn.Dropout(),
    torch.nn.Sigmoid(),
    torch.nn.Linear(256, 10),
)


loss_fn = torch.nn.MSELoss(reduction='sum')

In [14]:
B = 64
dtype=torch.float
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(3000):
    idx = np.random.randint(0,N,B)
    x = torch.tensor(training_set[idx], dtype=dtype).reshape(B,1,row_size,col_size)
    y = torch.tensor(list(map(lambda i: [1 if j == i else 0 for j in range(10)], training_labels[idx])), dtype=dtype)
    
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 0:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

0 102.8363037109375
100 22.15328025817871
200 13.929560661315918
300 9.717564582824707
400 9.56926155090332
500 8.08906078338623
600 6.679416656494141
700 7.0872650146484375
800 7.012990474700928
900 5.98568058013916
1000 5.203667163848877


KeyboardInterrupt: 

In [15]:
with torch.no_grad():
    x_test = torch.tensor(test_set, dtype=dtype).reshape(len(test_set),1,row_size,col_size)
    y_test = model(x_test).argmax(1)

In [16]:
(y_test.numpy() == test_labels).mean()

0.9831

In [17]:
np.where((y_test.numpy() == test_labels) == False)

(array([  62,  115,  211,  247,  259,  321,  340,  445,  447,  582,  619,
         674,  717,  740,  844,  846,  924,  926,  947,  956, 1014, 1039,
        1112, 1114, 1182, 1226, 1242, 1247, 1289, 1299, 1319, 1326, 1393,
        1500, 1530, 1549, 1553, 1569, 1641, 1681, 1709, 1722, 1737, 1754,
        1790, 1868, 1901, 2016, 2024, 2035, 2043, 2053, 2070, 2098, 2109,
        2129, 2130, 2135, 2182, 2185, 2266, 2293, 2387, 2406, 2454, 2462,
        2597, 2607, 2654, 2850, 2877, 2896, 2921, 2927, 2939, 2953, 2970,
        2995, 3005, 3030, 3060, 3073, 3132, 3289, 3422, 3503, 3520, 3534,
        3550, 3558, 3597, 3599, 3726, 3742, 3751, 3767, 3780, 3808, 3838,
        3853, 3893, 3906, 3941, 4075, 4078, 4199, 4205, 4224, 4238, 4256,
        4289, 4306, 4497, 4498, 4500, 4536, 4575, 4601, 4615, 4639, 4690,
        4783, 4807, 4814, 4837, 4860, 4874, 4886, 4956, 5159, 5288, 5331,
        5634, 5642, 5734, 5842, 5888, 5937, 5955, 5973, 6065, 6071, 6091,
        6166, 6505, 6597, 6625, 6651, 

In [34]:
test_sample = 8934
display(Image.fromarray(test_set[test_sample]))
print("ACTUAL: ", test_labels[test_sample])
print("PREDICTION: ", y_test[test_sample].item())

ACTUAL:  8
PREDICTION:  8
